**Note: This notebook must be changed by the student. As it is now, it does not approximate the solution to the Poisson problem!**

# 2D Solver for Electro-Static Problem on the Unit Square

In this exercise we write a solver for the 2D electro-static problem on the unit square, which can be shown to be equivalent to an $\mathcal{L}^1$ Hodge-Laplace problem.

\begin{align*}
    &\text{Find }\boldsymbol{E} \in D(\mathcal{L}^1) \\
    &\mathcal{L}^1\boldsymbol{E} = (\boldsymbol{curl}\ curl - \boldsymbol{grad}\ div)\boldsymbol{E} = -\boldsymbol{grad}\ \rho \quad \text{ in }\Omega=]0,1[^2,
\end{align*}
for the specific choice
\begin{align*}
    \rho(x, y) = 2\sin(\pi x)\sin(\pi y)
\end{align*}

## The Discrete Variational Formulation

The corresponding discrete variational formulation reads

\begin{align*}
    &\text{Find }\boldsymbol{E} \in V_h^1 \subset V^1 = H_0(curl;\Omega)\text{ such that } \\
    &a(\boldsymbol{E}, \boldsymbol{v}) = l(\boldsymbol{v})\quad\forall\ \boldsymbol{v}\in V_h^1
\end{align*}

where 
- $a(\boldsymbol{E},\boldsymbol{v}) \coloneqq \int_{\Omega} (curl\ \boldsymbol{E})(curl\ \boldsymbol{v}) + (\widetilde{div}_h\boldsymbol{E})(\widetilde{div}_h\boldsymbol{v}) ~ d\Omega$ ,
- $l(\boldsymbol{v}) := -\int_{\Omega} \boldsymbol{grad}\ \rho\cdot\boldsymbol{v} ~ d\Omega$.

## Discrete Model using de Rham objects

In [ ]:
from sympde.calculus      import dot
from sympde.topology      import elements_of, Square, Derham
from sympde.expr.expr     import BilinearForm, integral

domain  = Square('S', bounds1=(0, 1), bounds2=(0, 1))
derham  = Derham(domain, sequence=['h1', 'hcurl', 'l2'])

V0      = derham.V0
V1      = derham.V1
V2      = derham.V2

u0, v0  = elements_of(V0, names='u0, v0')
u1, v1  = elements_of(V1, names='u1, v1')
u2, v2  = elements_of(V2, names='u2, v2')

# bilinear forms corresponding to V0, V1 and V2 mass matrices
m0      = BilinearForm((u0, v0), integral(domain,      u0 * v0))
m1      = BilinearForm((u1, v1), integral(domain,      dot(u1, v1)))
m2      = BilinearForm((u2, v2), integral(domain,      u2 * v2))

# callable source term \rho
from sympy  import pi, sin, cos, lambdify, Tuple, Matrix
x,y             = domain.coordinates
rho             = 2*sin(pi*x)*sin(pi*y)
rho_lambdified  = lambdify((x, y), rho)

In [ ]:
from psydac.api.discretization  import discretize
from psydac.api.settings        import PSYDAC_BACKEND_GPYCCEL

backend = PSYDAC_BACKEND_GPYCCEL

In [ ]:
ncells  = [32, 32]  # Bspline cells
degree  = [4, 4]    # Bspline degree

In [ ]:
# discretize domain and derham
# <-
# <-

# define FEM spaces V0_h, V1_h and V2_h
#V0_h        = derham_h.V0
#V1_h        = derham_h.V1
#V2_h        = derham_h.V2

# Commuting projection operators
#P0, P1, P2  = derham_h.projectors()

# Exterior Derivative operators (grad and curl)
# <-

# Mass matrices
# <-
# <-
# <-

# <-
# <-
# <-

# Boundary Projectors
from utils                  import H1BoundaryProjector2D, HcurlBoundaryProjector2D
from psydac.linalg.basic    import IdentityOperator

#P_H1        = H1BoundaryProjector2D(V0, V0_h.vector_space)
#P_Hcurl     = HcurlBoundaryProjector2D(V1, V1_h.vector_space)

#I0          = IdentityOperator(V0_h.vector_space)
#I1          = IdentityOperator(V1_h.vector_space)

#P_H1_Gamma      = I0 - P_H1
#P_Hcurl_Gamma   = I1 - P_Hcurl

# Modified Operators for Projection Method (using P_H1, P_Hcurl, P_H1_Gamma, P_Hcurl_Gamma)
# <-
# ...
# <-

# Inverse M0 mass matrix
from psydac.linalg.solvers  import inverse
# <-

# System Matrix A
#A           = # <-

# rhs vector f
#rho_coeffs  = P0(rho_lambdified).coeffs
#f           = # <-

In [ ]:
import  time

tol     = 1e-10
maxiter = 1000

from utils import get_M1_block_kron_solver_2D
#pc          = get_M1_block_kron_solver_2D(V1_h.vector_space, ncells, degree, periodic=[False, False])
#M1_0_inv    = inverse(M1_0, 'pcg', pc=pc, tol=1e-11, maxiter=1000)
#A_inv       = inverse(A, 'pcg', pc=M1_0_inv, tol=tol, maxiter=maxiter)

t0      = time.time()
#E_h     = A_inv @ f
t1      = time.time()

## Computing the error norm

As the analytical solution is available, we want to compute the $L^2$ norm of the error.
In this example, the analytical solution is given by

$$
\boldsymbol{E}_{ex}(x, y) = -\frac{1}{\pi}\left(\begin{matrix} 
                \cos(\pi x) \sin(\pi y) \\
                \cos(\pi y) \sin(\pi x)
                \end{matrix}\right)
$$

In [ ]:
from psydac.fem.basic   import FemField

from sympde.expr        import Norm

E_ex            = Tuple( (-1/pi)*cos(pi*x) * sin(pi*y),
                         (-1/pi)*cos(pi*y) * sin(pi*x) )
#E_h_FemField    = FemField(V1_h, E_h)

error           = Matrix([u1[0] - E_ex[0], u1[1] - E_ex[1]])

# create the formal Norm object
l2norm          = Norm(error, domain, kind='l2')

# discretize the norm
#l2norm_h        = discretize(l2norm, domain_h, V1_h, backend=backend)

# assemble the norm
#l2_error        = l2norm_h.assemble(u1=E_h_FemField)

# print the result
#print( '> Grid          :: [{ne1},{ne2}]'.format( ne1=ncells[0], ne2=ncells[1]) )
#print( '> Degree        :: [{p1},{p2}]'  .format( p1=degree[0], p2=degree[1] ) )
#print( '> CG info       :: ',A_inv.get_info() )
#print( '> L2 error      :: {:.2e}'.format( l2_error ) )
#print( '' )
#print( '> Solution time :: {:.3g}'.format( t1-t0 ) )

## Visualization

We plot the true solution $\boldsymbol{E}_{ex}$, the approximate solution $\boldsymbol{E}_h$ and the error function $|\boldsymbol{E}_{ex} - \boldsymbol{E}_h|$.

In [ ]:
from utils import plot

E_ex_x   = lambdify((x, y), E_ex[0])
E_ex_y   = lambdify((x, y), E_ex[1])
#E_h_x    = E_h_FemField[0]
#E_h_y    = E_h_FemField[1]
#error_x  = lambda x, y: abs(E_ex_x(x, y) - E_h_x(x, y))
#error_y  = lambda x, y: abs(E_ex_y(x, y) - E_h_y(x, y))

if False:
    plot(gridsize_x     = 100, 
        gridsize_y     = 100, 
        title          = r'Approximation of Solution $\boldsymbol{E}$, first component', 
        funs           = [E_ex_x, E_h_x, error_x], 
        titles         = [r'$(\boldsymbol{E}_{ex})_1(x,y)$', r'$(\boldsymbol{E}_h)_1(x,y)$', r'$|(\boldsymbol{E}_{ex}-\boldsymbol{E}_h)_1(x,y)|$'],
        surface_plot   = True
    )

    plot(gridsize_x     = 100,
        gridsize_y     = 100,
        title          = r'Approximation of Solution $\boldsymbol{E}$, second component',
        funs           = [E_ex_y, E_h_y, error_y],
        titles         = [r'$(\boldsymbol{E}_{ex})_2(x,y)$', r'$(\boldsymbol{E}_h)_2(x,y)$', r'$|(\boldsymbol{E}_{ex}-\boldsymbol{E}_h)_2(x,y)|$'],
        surface_plot   = True
    )